In [39]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [40]:
from sklearn.datasets import load_svmlight_file
X_train, y_train = load_svmlight_file('data/agaricus.txt.train')
X_test, y_test = load_svmlight_file('data/agaricus.txt.test')

#

# 将svmliaht矩阵转换成稀疏矩阵

In [41]:

X_train.shape

(6513, 126)

In [42]:
model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5,
                          min_child_weight=1, gamma=0, subsample=0.8,
                          colsample_bytree=0.8, objective='binary:logistic',
                          nthread=4, scale_pos_weight=1, seed=27)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, nthread=4,
              num_parallel_tree=None, ...)

In [43]:
#计算准确率
y_pred = model.predict(X_test)
print("Accuracy: %.2f%%" % (accuracy_score(y_test, y_pred) * 100.0))

Accuracy: 100.00%


# GridSearchvc 搜索

In [44]:
from sklearn.model_selection import GridSearchCV

In [45]:

model = xgb.XGBClassifier(nthread=4)
param_grid = {
    'n_estimators': range(1, 51, 1),
    'max_depth': range(1, 10, 1)
}
clf = GridSearchCV(model, param_grid, scoring="accuracy", cv=5)  # 显式指定参数名
clf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, nthread=4,
                                     num_parallel_tree=None, ...),
             param_grid={'max_depth': range(1, 10),
                         'n_estimators': range(1, 51)},
             scoring='accuracy')

# early-stop

我们设置验证valid集，当我们迭代过程中发现在验证集上错误率增加，则提前停止迭代。

In [46]:
from sklearn.model_selection import train_test_split
X_train_part, X_validate, y_train_part, y_validate = train_test_split(X_train, y_train, test_size=0.3,random_state=0)

In [51]:
# 设置boosting迭代计算次数
num_round = 100

bst =xgb.XGBClassifier(max_depth=2, learning_rate=0.1, n_estimators=num_round, silent=True, objective='binary:logistic')

eval_set =[(X_validate, y_validate)]
bst.fit(X_train_part, y_train_part, early_stopping_rounds=10, eval_metric="error",
    eval_set=eval_set, verbose=True)

TypeError: fit() got an unexpected keyword argument 'early_stopping_rounds'

In [37]:
results = bst.evals_result()
results

XGBoostError: No evaluation result, `eval_set` is not used during training.

In [ ]:
results = bst.evals_result()
#print(results)

epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)

# plot log loss

plt.plot(x_axis, results['validation_0']['error'], label='Test')

plt.ylabel('Error')
plt.xlabel('Round')
plt.title('XGBoost Early Stop')
plt.show()

In [ ]:
# 设置boosting迭代计算次数
num_round = 100

# 没有 eraly_stop
bst =xgb.XGBClassifier(max_depth=2, learning_rate=0.1, n_estimators=num_round, silent=True, objective='binary:logistic')

eval_set = [(X_train_part, y_train_part), (X_validate, y_validate)]
bst.fit(X_train_part, y_train_part, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)

In [ ]:
results = bst.evals_result()
results

In [ ]:
# retrieve performance metrics
results = bst.evals_result()
#print(results)


epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)

# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

# plot classification error
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
plt.ylabel('Classification Error')
plt.title('XGBoost Classification Error')
plt.show()